In [1]:
import os 
import pandas as pd
import tensorflow as tf
import numpy as np


# Load the Dataset


In [4]:
dataset=pd.read_csv(
os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv'))

In [16]:
type(dataset)

pandas.core.frame.DataFrame

In [5]:
dataset.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [10]:
dataset[dataset['insult']==1].tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0
159554,ffbdbb0483ed0841,and i'm going to keep posting the stuff u dele...,1,0,1,0,1,0


# Let's Pre-Process the Data

In [6]:
from tensorflow.keras.layers import TextVectorization

In [7]:
TextVectorization??

In [8]:
x=dataset['comment_text']
y=dataset[dataset.columns[2:]].values

In [10]:
max_words=200000
vectorizer=TextVectorization(max_tokens=max_words,
                            output_sequence_length=1500,
                            output_mode='int')

In [11]:
vectorizer.adapt(x.values)

In [8]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [19]:
vectorizer('the to of and')[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([2, 3, 4, 5], dtype=int64)>

In [18]:
vectorizer_text=vectorizer(x.values)

In [10]:
vectorizer_text

<tf.Tensor: shape=(159571, 1500), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [19]:
data = tf.data.Dataset.from_tensor_slices((vectorizer_text,y))
data=data.cache()
data=data.shuffle(160000)
data=data.batch(16)
data=data.prefetch(8)

In [20]:
batch_x,batch_y=data.as_numpy_iterator().next()

In [21]:
batch_x.shape

(16, 1500)

In [22]:
train=data.take(int(len(data)*.7))
val=data.skip(int(len(data)*.7)).take(int(len(data)*.2))
test=data.skip(int(len(data)*.9)).take(int(len(data)*.1))

In [20]:
train.as_numpy_iterator().next()

(array([[ 2017,     2,    28, ...,     0,     0,     0],
        [ 1115,     2,    24, ...,     0,     0,     0],
        [78203,   145,    24, ...,     0,     0,     0],
        ...,
        [   38,     9,     6, ...,     0,     0,     0],
        [  104,    61,    19, ...,     0,     0,     0],
        [   40,    33,     7, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [22]:
len(test)


997

# Create Sequential Model

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional,Dense,Embedding

In [28]:
model=Sequential()
model.add(Embedding(max_words+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))


In [29]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [31]:
history=model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 4539s 649ms/step - loss: 0.0632 - val_loss: 0.0480


In [32]:
history.history

{'loss': [0.06321804970502853], 'val_loss': [0.04804793372750282]}

In [34]:
model_version=1
model.save(f"../trained/{model_version}")

INFO:tensorflow:Assets written to: ../trained/1\assets


INFO:tensorflow:Assets written to: ../trained/1\assets


In [35]:
model.save("toxic_classifier.h5")

In [9]:
input=vectorizer("I hate you")

In [12]:
model.predict(np.expand_dims(input,0))

1/1 [==============================] - 2s 2s/step


array([[0.5944195 , 0.00254682, 0.11824892, 0.00525489, 0.16501401,
        0.02532417]], dtype=float32)

In [38]:
dataset.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [12]:
from tensorflow import keras
model= keras.models.load_model("toxic_classifier.h5")

In [24]:
batch_x,batch_y=test.as_numpy_iterator().next()

In [13]:
(model.predict(np.expand_dims(input,0))>0.5).astype(int)

1/1 [==============================] - 0s 144ms/step


array([[1, 0, 0, 0, 0, 0]])

In [26]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

# Evaluating our Model

In [35]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy, Accuracy

In [36]:
precision=Precision()
re=Recall()
cat=CategoricalAccuracy()
ac=Accuracy()

In [37]:
for batch in test.as_numpy_iterator():
    x_true,y_true=batch
    yhat=model.predict(x_true)
    y_true=y_true.flatten()
    yhat=yhat.flatten()
    precision.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    cat.update_state(y_true,yhat)
    ac.update_state(y_true,yhat)
    

1/1 [==============================] - 0s 122ms/step


1/1 [==============================] - 0s 127ms/step


1/1 [==============================] - 0s 124ms/step


1/1 [==============================] - 0s 120ms/step


In [38]:
print(f'Precision: {precision.result().numpy()},Recall :{re.result().numpy()}, CategoricalAccuracy : {cat.result().numpy()}, accuracy : {ac.result().numpy()}' )

Precision: 0.8629462718963623,Recall :0.6015446186065674, CategoricalAccuracy : 0.4704112410545349, accuracy : 0.0


In [39]:
!pip install gradio jinja2

     -------------------------------------- 15.8/15.8 MB 553.0 kB/s eta 0:00:00
     ---------------------------------------- 45.7/45.7 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 101.4/101.4 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 57.8/57.8 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 202.8/202.8 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 71.5/71.5 kB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 56.4/56.4 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 644.2 kB/s eta 0:00:00
     -------------------------------------- 50.5/50.5 kB 874.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 199.2/199.2 kB 1.3 MB/s eta 0:00:00
     -------------

In [13]:
import gradio as gr

In [14]:
def predict(comment):
    vectorized=vectorizer([comment])
    res=model.predict(vectorized,0)
    text=''
    for idx,col in enumerate(dataset.columns[2:-1]):
        text+='{}: {}\n'.format(col, res[0][idx]>0.5)
    return text

NameError: name 'vectorizer' is not defined

In [28]:
interface=gr.Interface(fn=predict,inputs=gr.inputs.Textbox(lines=2,placeholder='Comment to check'),
                      outputs='text')

In [ ]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
